In [33]:
import getpass
import os
import re
from typing import Dict, List, TypedDict

import sympy as sp
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, MessagesState, StateGraph


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

llm = ChatOpenAI(model='gpt-4o-mini-2024-07-18', temperature=0)

In [34]:
class AgentState(TypedDict):
    messages: List[MessagesState]
    task: str
    plan_string: str
    steps: List
    results: Dict
    result: str


# Define a sympy-based tool
@tool
def calculate(expression: str) -> str:
    """Calculate an arithmetic expression using sympy.

    Args:
        expression: arithmetic expression as a string (e.g., '3 + 4')

    Returns:
        The evaluated result as a string.
    """
    try:
        expr = sp.sympify(expression)
        evaluated = expr.evalf() if expr.is_number else expr
        return str(evaluated)
    except Exception as e:
        return f"Error: {e}"

# Define a LLM tool
@tool
def llm_tool(expression: str) -> str:
    """Use LLM like yourself to process the input string.
    This is useful for tasks that require reasoning or understanding of the context.
    For example, you can use it to provide explanations.

    Args:
        expression: input string to be processed by LLM

    Returns:
        The processed output as a string.
    """
    response = llm.invoke([SystemMessage(content=expression)])
    return response.content


# Augment the LLM with the sympy tool
tools = [calculate, llm_tool]
tools_by_name = {tool.name: tool for tool in tools}

In [35]:
planer_prompt_system = """
### INSTRUCTIONS ###
You are a math problem solving planner. For the following task, make plans that can solve \
the problem step by step. For each plan, indicate which external tool together with tool input to retrieve \
evidence. You can store the evidence into a variable #E that can be called by later tools \
(Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Calculate[input]: A tool that is used for solving math expressions using sympy.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and \
common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.

### EXAMPLE ###
Task: A person has $500. They spend 40% on groceries, 20% on utilities, and 10% on transportation. \
How much money do they have left?

Plan: Calculate the total amount spent on groceries, utilities, and transportation.
#E1 = Calculate[0.40 * 500 + 0.20 * 500 + 0.10 * 500]

Plan: Subtract the total amount spent from the initial amount to find the remaining money.
#E2 = Calculate[500 - #E1]
"""

planer_prompt_human = """
### YOUR TASK ###
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}
Plan:
"""

planer_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", planer_prompt_system),
        ("user", planer_prompt_human)
    ]
)

solver_prompt = """
### INSTRUCTIONS ###
Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

Respond with the answer in a format: "Answer: <value>". Value should be a number without \
any units. If you are not sure about the answer, respond with "I don't know".

### TASK ###
{task}

### PLAN ###
{plan}

### ANSWER ###
"""

In [ ]:
# Planner node
def planner(state: AgentState):
    """Generate a step-by-step plan to solve the problem"""
    regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"

    task = state["task"]

    prompt = planer_prompt_template.format_messages(task=task)
    response = llm.invoke(state['messages'] + prompt)

    # Extract the plan string from the response
    matches = re.findall(regex_pattern, response.content)
    return {"steps": matches, "plan_string": response.content, "messages": state["messages"] + [response]}

def _get_current_task(state: AgentState):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1

# Executor node
def executor(state: AgentState):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool_name, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    if tool_name == "Calculate":
        result = calculate.invoke(tool_input)
    elif tool_name == "LLM":
        result = llm_tool.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)

    tool_message = ToolMessage(content=f"{tool_input}\nResult: {result}", artifact=result, tool_call_id=step_name)

    return {"results": _results, "messages": state["messages"] + [tool_message]}

def solve(state: AgentState):
    plan = ""
    for _plan, step_name, tool_name, tool_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool_name}[{tool_input}]"
    prompt = solver_prompt.format(plan=plan, task=state["task"])
    result = llm.invoke(prompt)
    return {"result": result.content, "messages": state["messages"] + [result]}

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading a math problems solution."
            " Generate critique and recommendations for the user's solution path."
            " Make sure to be specific and provide detailed feedback."
            " Do not refer to the solultion clarification or style, just focus on the solution path."
            " If the solution is correct, just say 'END' with upper case."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def reflection_node(state: AgentState) -> AgentState:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage, "tool": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = llm.invoke(
        reflection_prompt.format_messages(messages=translated),
        stop=["\n\n###"],
    )

    if res.content.lower == "correct":
        # If the solution is correct, we can stop
        return {"messages": [HumanMessage(content=res.content)]}

    # We treat the output of this as human feedback for the generator
    return {"messages": state["messages"] + [HumanMessage(content=res.content)]}


def _route(state: AgentState):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

def should_continue(state: AgentState):
    if len([msg for msg in state["messages"] if isinstance(msg, AIMessage)]) > 6:
        print("Reached the limit of messages")
        return END

    if "END" in state["messages"][-1].content:
        # If the last message is "END", we stop
        print("Reflection node returned END")
        return END

    return "plan"

graph = StateGraph(AgentState)

graph.add_node("plan", planner)
graph.add_node("tool", executor)
graph.add_node("solve", solve)
graph.add_node("reflect", reflection_node)

graph.add_edge(START, "plan")
graph.add_edge("plan", "tool")
graph.add_conditional_edges("tool", _route)
graph.add_edge("solve", "reflect")
graph.add_conditional_edges("reflect", should_continue)



agent = graph.compile()


task = "James has a $1000 budget. He spends 30% on food, \
15% on accommodation, 25% on entertainment. \
How much does he spend on coursework materials?"

messages = [HumanMessage(content=task)]

# # Print full conversation with steps
# for m in agent.stream({"task": task, "messages": messages, "should_continue": True}):
#     print(m)

result = agent.invoke({"task": task, "messages": messages})

# Print full conversation with steps
for m in result["messages"]:
    m.pretty_print()

Reflection node returned END
================================ Human Message =================================

James has a $1000 budget. He spends 30% on food, 15% on accommodation, 25% on entertainment. How much does he spend on coursework materials?
================================== Ai Message ==================================

Plan: First, I will calculate the total amount James spends on food, accommodation, and entertainment by determining the percentages of his $1000 budget allocated to each category. This will involve calculating 30% for food, 15% for accommodation, and 25% for entertainment. I will sum these amounts to find the total expenditure on these categories. 
#E1 = Calculate[0.30 * 1000 + 0.15 * 1000 + 0.25 * 1000]

Plan: Next, I will subtract the total amount spent on food, accommodation, and entertainment from James's initial budget of $1000 to find out how much he has left for coursework materials. 
#E2 = Calculate[1000 - #E1]
================================= Tool